In [1]:

import reflexgenerator.sources
from reflexgenerator.sources import (
    Register, Mask, PinMap_from_json, Metadata, Collection)
from reflexgenerator.io import load

from pathlib import Path
import pandas as pd
from reflexgenerator.generator.xref import UidReference, make_anchor, create_link, filter_refs_by_type


In [2]:
fw_target = r"schema\example.yml"
io_target = r"schema\ios.yml"

In [3]:
device = load(fw_target)
ios = load(io_target)

In [4]:
metadata = Metadata(
    **{"device": device["device"],
    "whoAmI": device["whoAmI"],
    "architecture": device["architecture"],
    "firmwareVersion": device["firmwareVersion"],
    "hardwareTargets": device["hardwareTargets"]})
print(metadata)


Metadata(device='ExampleDevice', whoAmI=0, firmwareVersion='0.1', hardwareTargets='0.1', architecture='atmega', uid=ref-Device-Metadata-ExampleDevice_0)


In [5]:
# build masks
if "bitMasks" in device:
    bitMasks = Collection([Mask.from_json(mask) for mask in device["bitMasks"].items() if mask is not None])
    bitMasks_df = pd.DataFrame([mask.to_dict() for mask in bitMasks])
    bitMasks_df.head(5)
else:
    bitMasks = None
    bitMasks_df = pd.DataFrame()

if "groupMask" in device:
    groupMasks = Collection([Mask.from_json(mask) for mask in device["groupMask"].items() if mask is not None])
    groupMasks_df = pd.DataFrame([mask.to_dict() for mask in groupMasks])
    groupMasks_df.head(5)
else:
    groupMasks = None
    groupMasks_df = pd.DataFrame()

In [6]:
if "registers" in device:
    regs = Collection([Register.from_json(reg) for reg in device["registers"].items() if reg is not None])
    register_df = pd.DataFrame([reg.to_dict() for reg in regs])
    register_df.head(5)
else:
    regs = None
    register_df = pd.DataFrame()

In [7]:
pinMapping = Collection([PinMap_from_json(pinmap) for pinmap in ios.items() if pinmap is not None])
pinMapping_df = pd.DataFrame([pinmap.to_dict() for pinmap in pinMapping])
pinMapping_df.head(5)


,name,port,pinNumber,direction,pinMode,triggerMode,interruptPriority,interruptNumber,description,uid,allowRead,initialState,invert
0,Lick0,PORTA,0,[input](#ref-DirectionType-input),[tristate](#ref-InputPinModeType-tristate),[toggle](#ref-TriggerModeType-toggle),[low](#ref-InterruptPriorityType-low),0.0,Input for the Lick 0.,ref-Device-InputPin-Lick0,NaN,NaN,NaN
1,Lick1,PORTA,1,[input](#ref-DirectionType-input),[tristate](#ref-InputPinModeType-tristate),[toggle](#ref-TriggerModeType-toggle),[low](#ref-InterruptPriorityType-low),1.0,Input for the Lick 1.,ref-Device-InputPin-Lick1,NaN,NaN,NaN
2,Valve0,PORTA,6,[output](#ref-DirectionType-output),[wiredOr](#ref-OutputPinModeType-wiredOr),NaN,NaN,NaN,Controls the transistor of Valve 0.,ref-Device-OutputPin-Valve0,False,[low](#ref-InitialStateType-low),False


In [8]:
payloadMembers = Collection(list(filter_refs_by_type(reflexgenerator.sources.PayloadMember).values()))
payloadMembers_df = pd.DataFrame([payloadMember.parent.to_dict() for payloadMember in payloadMembers])
payloadMembers_df.head(5)

,name,mask,offset,maskType,description,converter,defaultValue,maxValue,minValue,interfaceType,uid
0,ADC,None,0,None,The value of the board ADC.,None,None,None,None,None,ref-Device-PayloadMember-ADC
1,Encoder,None,1,None,The value of the quadrature counter in Port 2.,None,None,None,None,None,ref-Device-PayloadMember-Encoder


In [9]:
register_df_copy = register_df.copy()
for i in register_df_copy.index:
    if register_df_copy.at[i, 'maskType']:
        if isinstance(register_df_copy.at[i, 'maskType'][0], Mask):
            register_df_copy.at[i, 'maskType'] = [x.uid.render_pointer() for x in register_df_copy.at[i, 'maskType']]

bitMasks_df_copy = bitMasks_df.copy()
for i in bitMasks_df_copy.index:
    bitMasks_df_copy.at[i, 'name'] = bitMasks_df_copy.at[i, 'uid'].render_pointer()

groupMasks_df_copy = groupMasks_df.copy() if groupMasks_df is not None else pd.DataFrame()
for i in groupMasks_df_copy.index:
    groupMasks_df_copy.at[i, 'name'] = groupMasks_df_copy.at[i, 'uid'].render_pointer()

payloadMembers_df_copy = payloadMembers_df.copy() if payloadMembers_df is not None else pd.DataFrame()
for i in payloadMembers_df_copy.index:
    payloadMembers_df_copy.at[i, 'name'] = payloadMembers_df_copy.at[i, 'uid'].render_pointer()


In [10]:
def format_table(df: pd.DataFrame) -> str:
    return df.loc[:, df.columns != "uid"].to_markdown(index=False, tablefmt="pipe")

In [11]:
metadata_format = ("".join([f'''<font size="4"> - {k}: {v} </font> \n\n''' for k,v in metadata.to_dict().items() if k != "uid"]))

txt = f"""

# Device

{metadata_format}
--------

# Registers

## Summary table
{format_table(register_df_copy)}

## Technical documentation
{"".join([reg.format_dict() for reg in regs])}
--------

# BitMasks

## Summary table
{format_table(bitMasks_df_copy)}

## Technical documentation
{"".join([mask.format_dict() for mask in bitMasks])}
--------
# PayloadMembers

## Summary table
{format_table(payloadMembers_df_copy)}

## Technical documentation
{"".join([payloadMember.format_dict() for payloadMember in payloadMembers])}

# IOs
{format_table(pinMapping_df)}


## References
{reflexgenerator.sources.PayloadType.format_anchor_references()}

{reflexgenerator.sources.RegisterType.format_anchor_references()}

{reflexgenerator.sources.VisibilityType.format_anchor_references()}

{reflexgenerator.sources.MaskCategory.format_anchor_references()}

{reflexgenerator.sources.DirectionType.format_anchor_references()}

{reflexgenerator.sources.InputPinModeType.format_anchor_references()}

{reflexgenerator.sources.TriggerModeType.format_anchor_references()}

{reflexgenerator.sources.InterruptPriorityType.format_anchor_references()}

{reflexgenerator.sources.OutputPinModeType.format_anchor_references()}

{reflexgenerator.sources.InitialStateType.format_anchor_references()}



"""







AttributeError: 'UidReference' object has no attribute 'format_dict'

{"### DO\n> description = Bitmask representing the state of the digital outputs. \n\n> value = None \n\n> bits = [BitOrValue(name='DO0', value='0x1', description='The state of digital output pin 0.', uid=ref-Device-BitOrValue-DO0), BitOrValue(name='DO1', value='0x2', description='The state of digital output pin 1.', uid=ref-Device-BitOrValue-DO1)] \n\n> maskCategory = [BitMask](#ref-MaskCategory-BitMask) \n\n"}

In [ ]:
with open("documentation.md", "w") as text_file:
    text_file.write(txt)

import markdown
with open("documentation.html", "w") as text_file:
    text_file.write(markdown.markdown(txt, tab_length=4, extensions=['extra', 'smarty', 'sane_lists']))